In [ ]:
import fitz  # PyMuPDF
import re
from collections import defaultdict
import json
import operators
import base64

In [ ]:
# operators.voice.readoutload(
#   "The digital code of DNA..",
#   write_address="nowgeneratedspeechforstudy.wav",
#   spkr_id=24,
#   samplerat=16000)

In [ ]:
# Open the PDF file
doc = fitz.open("The Digital Code of DNA.pdf")  # Replace with the actual PDF file path

segments = []
current_segment = {"text": "", "style": None, "image": None}

# Process each page in the document
for page in doc:
    blocks = page.get_text("dict")["blocks"]
    for block in blocks:
        if "lines" in block:
            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    text = span["text"]
                    style = {"font": span["font"], "size": span["size"], "color": span["color"]}
                    current_segment = {"text": text.strip(), "style": style}
                    segments.append(current_segment)
        elif "image" in block:
            print("there is an image")
            
            base64_image = block["image"]
            image_data_str = base64.b64encode(base64_image).decode('utf-8')
            image_data_html = f"data:image/jpeg;base64,{image_data_str}"
            block["image"] = image_data_html
            
            current_segment = {"text": "", "style": '', "image": block}
            segments.append(current_segment)
# Don't forget to add the last segment
if current_segment["text"]:
    segments.append(current_segment)
    
# Close the document
doc.close()

# Example output
for segment in segments[10:13]:  # Display the first few segments for brevity
    print(f"Text: {segment['text']}\nsegments: {segment['text'].split('.')}\nStyle: {segment['style']}\n---\n")


In [ ]:
segments

### mark down

In [ ]:
def flag_text(text, style):
  if style['size'] >= 20: return 'large'
  elif style['size'] < 20 and style['size'] >= 10: return 'heading'
  elif style['size'] < 10 and style['size'] >= 9: return 'small'
  elif style['size'] < 9 and style['size'] > 8: return 'text'
  else: return "span"

print(flag_text("Tools to modify DNA", {"size": 9.5}))
print(flag_text("The enzymes that function in cells to copy, cut and join DNA molecules were also exploited as key tools for revolutionary new tech- niques in molecular biology, including the cloning of genes and expression of their proteins, and mapping the location of genes on chromosomes. The ability to recreate the process of DNA replication artificially in the laboratory led to the development of two techniques", {
  "size": 8.835000038146973}))
print(flag_text("2", {"size": 5.301000118255615}))


In [ ]:
for segment in segments:
  if "image" in segment: segment['flag'] = "image"
  else: segment['flag'] = flag_text(segment['text'], segment['style'])

### merge the single character large text that is the first letter of the following text

In [ ]:
# Creating a new list with the items that should remain
new_segments = []
for i in range(len(segments)):
    # identify the large single char segments and deny them
    if (segments[i]["flag"] == "large" and len(segments[i]["text"]) == 1 and segments[i+1]["flag"] == "text"): print(segments[i]["text"])
    elif (segments[i]["flag"] == "text" and len(segments[i]["text"]) > 1 and len(segments[i-1]["text"]) == 1 and segments[i-1]["flag"] == "large"):
        segments[i]["text"] = segments[i-1]["text"]+segments[i]["text"]
        new_segments.append(segments[i])
    else: new_segments.append(segments[i])

segments = new_segments


### remove the small spans inside text
which is actually a marker or flag but can't handle that right now

In [ ]:
# Creating a new list with the items that should remain
new_segments = []
for i in range(len(segments)):
    if segments[i]['flag'] == "image": print("image:", i)
    if not (segments[i]['flag'] == "span" and i > 0 and segments[i-1]['flag'] == 'text'):
        new_segments.append(segments[i])

segments = new_segments


```TODO:```  merge spans that are in the middle of two segments of equal style

### merge segments of the same style

In [ ]:
while True:
    global segments
    new_segments = []
    i = 0
    while i < len(segments) - 1:
        if ("image" in segments[i] or "image" in segments[i+1]):
            new_segments.append(segments[i])
            i+=1
        else:

            if segments[i]['style']['size'] == segments[i+1]['style']['size']:
                segments[i]['text'] = segments[i]['text'] +' '+ segments[i+1]['text']
                new_segments.append(segments[i])
                i += 2
            else:
                new_segments.append(segments[i])
                i += 1

    # Handle the last segment if it wasn't merged
    if i == len(segments) - 1:
        new_segments.append(segments[i])

    print(len(new_segments))
    if (len(segments) == len(new_segments)): break
    else: segments = new_segments



## Add Id

In [ ]:
id_counter = 0
for segment in segments:
  segment['id'] = id_counter
  id_counter += 1

### generate summeries

In [ ]:
for segment in segments:
  if (segment['flag'] != 'span' and len(segment['text'])>20):
    print(segment['id'])
    segment['summery'] = operators.summary.summerize(segment['text'])
    print(segment['summery'])

```TODO:``` config summery generation for splits(which has to be configed to smaller length)

### Split to smaller chonks

In [ ]:
def replace_dots_with_commas(text):
    # Define a regular expression pattern to match decimal numbers
    pattern = r'\b(\d+)\.(\d+)\b'

    # Replace dots with commas
    replaced_text = re.sub(pattern, r'\1,,,\2', text)
    
    return replaced_text

def convert_commas_back_to_dots(text):
    # Define a regular expression pattern to match numbers with commas
    pattern = r'\b(\d+),,,(\d+)\b'

    # Convert commas back to dots
    converted_text = re.sub(pattern, r'\1.\2', text)
    
    return converted_text

def chop_down(text):
  # remove some dots that are not meant as end of text
  text = text.replace("(ref. ", "(ref ")
  text = text.replace("(Fig. ", "(Fig ")
  text = text.replace("(Fig. ", "(Fig ")

  subtexts = replace_dots_with_commas(text).split('.')
  for i in range(len(subtexts)):
    subtexts[i] += '.'
    subtexts[i] = convert_commas_back_to_dots(subtexts[i])
  
  filtered_array = [s for s in subtexts if s != "."]

  return filtered_array


In [ ]:
for segment in segments:
  if segment['flag'] != 'span':
    segment['parts'] = chop_down(segment['text'])
    if 'summery' in segment:
      segment['summery_parts'] = chop_down(segment['summery'])

## generate voices

In [ ]:
# for the summery texts
for segment in segments:
  if 'summery_parts' in segment:
    # main part:
    thesummarysplitted = [substr for substr in re.split(r"[.!?;:]", segment['summery']) if substr]
    print(segment['id'])
    print(thesummarysplitted)
    for tmptext in thesummarysplitted:
      if len(tmptext) > 1: operators.voice.readoutload(tmptext+'.', f"./voices/{segment['id']}.summary.wav", False, spkr_id=24)

    # sub parts
    if len(segment['summery_parts'])>1:
      index = 0
      for summery_part in segment['summery_parts']:
        print(segment['id'], index)
        thesummarysplitted = [substr for substr in re.split(r"[!?;:]", summery_part) if substr]
        print(thesummarysplitted)
        for tmptext in thesummarysplitted:
          if len(tmptext) > 1: operators.voice.readoutload(tmptext+'.', f"./voices/{segment['id']}.summary_part.{index}.wav", False, spkr_id=24)
        # operators.readoutload(segment['text'], f"./voices/{segment['id']}.main.wav", False)
        index+=1

In [ ]:
# for the original texts
for segment in segments:
  if 'parts' in segment:
    # main part:
    thesummarysplitted = [substr for substr in re.split(r"[.!?;:]", segment['text']) if substr]
    print(segment["id"])
    print(thesummarysplitted)
    for tmptext in thesummarysplitted:
      if len(tmptext) > 1: operators.voice.readoutload(tmptext+'.', f"./voices/{segment['id']}.original.wav", False, spkr_id=24)

    # sub parts
    if len(segment['parts'])>1:
      index = 0
      for summery_part in segment['parts']:
        print(segment['id'], index)
        thesummarysplitted = [substr for substr in re.split(r"[!?;:]", summery_part) if substr]
        print(thesummarysplitted)
        for tmptext in thesummarysplitted:
          if len(tmptext) > 1: operators.voice.readoutload(tmptext+'.', f"./voices/{segment['id']}.part.{index}.wav", False, spkr_id=24)
        # operators.readoutload(segment['text'], f"./voices/{segment['id']}.main.wav", False)
        index+=1

# Store

In [ ]:
with open('./output.json', 'w') as json_file:
    json.dump(segments, json_file, indent=4)
with open('./output_user_modified.json', 'w') as json_file:
    json.dump(segments, json_file, indent=4)

```TODO:``` have AI understand what each part is
for example: one segment might be author, another maybe the main context